In [1]:
import os
import csv
import logging
from dotenv import load_dotenv

In [2]:
# 設置日誌
logging.basicConfig(filename='grading.log', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

load_dotenv()
HOMEWORK_DIR = os.path.abspath(os.getenv('HOMEWORK_DIR'))

# 確認路徑是否存在
if not os.path.exists(HOMEWORK_DIR):
    raise FileNotFoundError(f"The directory {HOMEWORK_DIR} does not exist. Current path: {HOMEWORK_DIR}")

os.environ["ANTHROPIC_API_KEY"] = os.getenv('ANTHROPIC_API_KEY')
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
use_anthropic = True

with open('assignment_requirements.txt', 'r', encoding='utf-8') as f:
    assignment_requirements = f.read()

In [3]:
SystemPrompt = '''
你是一位公正的教育者，負責根據指定的評分標準評估學生的作業。
對於每位學生，請根據其作業質量和對作業要求的遵守程度，給出100分制的分數。
遵守配分原則：
每小題7分
應該準確依照每題的正確性給予分數
每小題的分數範圍是0-7分
不應該出現空白卻有分數的情況
請確保分數稍微偏高一些。同時，請用30個字以內的繁體中文提供簡短評分原因，說明評分理由。

請嚴格按照以下格式輸出，每位學生一行：
學號,分數,原因

注意事項：
1. 不要添加任何額外的文字或信息
2. 每行必須包含且僅包含：學號、分數、評語，用逗號分隔
3. 分數必須是0-100的整數
4. 原因必須是繁體中文，不超過30字
5. 不要使用引號或其他標點符號包裹內容

範例輸出格式：
411234567,85,作業內容完整，理解深入
411234568,92,解釋清晰，舉例恰當
'''

In [4]:
from main import process_file
# 遍歷學生資料夾
students = []
for student_folder in os.listdir(HOMEWORK_DIR):
    student_path = os.path.join(HOMEWORK_DIR, student_folder)
    if os.path.isdir(student_path):
        try:
            # 處理資料夾名稱編碼問題
            try:
                folder_name = student_folder
                if not isinstance(folder_name, str):
                    # 如果不是字符串，嘗試解碼
                    encodings = ['utf-8', 'big5', 'gbk', 'latin1']
                    for encoding in encodings:
                        try:
                            folder_name = folder_name.decode(encoding)
                            break
                        except (UnicodeDecodeError, AttributeError):
                            continue
                    else:
                        logging.error(f"無法解碼資料夾名稱: {student_folder}")
                        continue
                
                # 分割學號和姓名
                parts = folder_name.split('_', 1)
                if len(parts) != 2:
                    logging.warning(f"無效的資料夾名稱格式: {folder_name}")
                    continue
                student_id, student_name = parts
            except Exception as e:
                logging.error(f"處理資料夾名稱時發生錯誤: {student_folder}, 錯誤: {str(e)}")
                continue
        except ValueError:
            logging.warning(f"無效的學生文件夾名稱: {student_folder}")
            continue
        
        if student_folder.endswith('無附件'):
            students.append({
                'id': student_id,
                'name': student_name.replace('無附件',''),
                'images': [],
                'texts': [],
                'path': student_path,
                'has_attachments': False
            })
        else:
            # Process student files
            try:
                if not os.path.exists(student_path):
                    logging.error(f"學生目錄不存在: {student_path}")
                    students.append({
                        'id': student_id,
                        'name': student_name,
                        'images': [],
                        'texts': [],
                        'path': student_path,
                        'has_attachments': False
                    })
                    continue

                files = os.listdir(student_path)
                if not files:
                    logging.warning(f"學生目錄為空: {student_path}")
                    students.append({
                        'id': student_id,
                        'name': student_name,
                        'images': [],
                        'texts': [],
                        'path': student_path,
                        'has_attachments': False
                    })
                    continue

                images = []
                texts = []
                
                for f in files:
                    try:
                        file_path = os.path.join(student_path, f)
                        process_file(file_path, images, texts, depth=0, max_depth=10)
                    except Exception as e:
                        logging.error(f"處理文件失敗 {file_path}: {str(e)}")
                        continue
                
                students.append({
                    'id': student_id,
                    'name': student_name,
                    'images': images,
                    'texts': texts,
                    'path': student_path,
                    'has_attachments': len(images) > 0 or len(texts) > 0
                })
            except Exception as e:
                logging.error(f"處理學生目錄失敗 {student_path}: {str(e)}")
                students.append({
                    'id': student_id,
                    'name': student_name,
                    'images': [],
                    'texts': [],
                    'path': student_path,
                    'has_attachments': False
                })

logging.info(f"總共找到 {len(students)} 位學生的作業")

In [5]:
from main import grade_batch_assignments
# 開始評分
output_file_name = HOMEWORK_DIR.split('/')[-1].split('_')[1]
with open(f'{output_file_name}.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['學號', '姓名', '分數', '評語']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    
    for i in range(0, len(students), 10):
        batch = students[i:i+10]
        grades = grade_batch_assignments([s for s in batch if s['has_attachments']], use_anthropic=use_anthropic)
        
        for student in batch:
            if not student['has_attachments']:
                writer.writerow({'學號': student['id'], '姓名': student['name'], '分數': 0, '評語': '未繳交'})
                print({'學號': student['id'], '姓名': student['name'], '分數': 0, '評語': '未繳交'})
            else:
                grade = next((g for g in grades if g['id'] == student['id']), None)
                if grade:
                    writer.writerow({'學號': student['id'], '姓名': student['name'], '分數': grade['score'], '評語': grade['comment']})
                    print({'學號': student['id'], '姓名': student['name'], '分數': grade['score'], '評語': grade['comment']})
                else:
                    writer.writerow({'學號': student['id'], '姓名': student['name'], '分數': 0, '評語': '讀取異常'})
                    print({'學號': student['id'], '姓名': student['name'], '分數': 0, '評語': '讀取異常'})
                    logging.warning(f"{student['id']}: 讀取異常")

print("評分完成。請檢查 grading.log 文件以獲取詳細的錯誤信息。")


{'學號': '411770356', '姓名': '李捷媮', '分數': '90', '評語': '回答正確，邏輯回歸模型建立和預測適當，惟解釋稍嫌簡略。'}
{'學號': '411770893', '姓名': '蔡元祐_', '分數': 0, '評語': '未繳交'}
{'學號': '412777269', '姓名': '江慶澤', '分數': '65', '評語': '程式理解良好但代碼未執行，數據格式錯誤，缺少完整回答'}
{'學號': '411770711', '姓名': '謝依恬', '分數': '82', '評語': '理解線性迴歸概念，但題目理解有誤，預測方向相反'}
{'學號': '411771073', '姓名': '徐杰弘', '分數': '75', '評語': '回歸方程式正確，但方法簡單且無統計學檢驗，第二題未完成。'}
{'學號': '410770548', '姓名': '鄭煜霖', '分數': '60', '評語': '答案不完整，缺乏第一題與第二題的實作結果'}
{'學號': '411770273', '姓名': '何長均', '分數': '62', '評語': '程式嘗試良好但有錯誤，未完整回答問題，缺少數據分析和模型建立'}
{'學號': '412777095', '姓名': '謝宗曄', '分數': '88', '評語': '邏輯迴歸計算大致正確，但數據處理有誤。能理解模型概念與應用，表達清晰。'}
{'學號': '411770737', '姓名': '鍾鎮安', '分數': '75', '評語': '作業未完成，僅有部分sigmoid函數計算。'}
{'學號': '411770471', '姓名': '蔡函庭', '分數': '85', '評語': '程式格式清晰，第一題完整，但第二題未完成實作。'}
讀取 notebook 時發生錯誤 (編碼: utf-8): Notebook does not appear to be JSON: '# -*- coding: utf-8 -*-\n"""\nSpyder Ed...
讀取 notebook 時發生錯誤 (編碼: latin1): Notebook does not appear to be JSON: '# -*- coding: utf-8 -*-\n"""\nSpy